In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../aipowerdatasetconstruction/'))

from image_util import labeledImage
from azureapi import AzureCVObjectDetectionAPI
from sklearn.model_selection import train_test_split

In [ ]:
molecules_dir = './elsevier_images/train/image/'
labels = ['molecule', 'microscopic_image', 'diagram']
labels_dir = ['./elsevier_images/train/' + l + '_labels' for l in labels]

labeled_images = []

for file in os.listdir(molecules_dir):
    if file.endswith(".jpeg"):
        image_path = os.path.join(molecules_dir, file)
        labeled_images.append(labeledImage(image_path))
        
        for i in range(len(labels)):
            label_path = os.path.join(labels_dir[i], file.split('.')[0] + '.txt')
            if os.path.exists(label_path):
                #print(label_path)
                labeled_images[-1].add_labels_from_file(labels[i], label_path)

In [ ]:
Endpoint    = 'https://westus2.api.cognitive.microsoft.com/'
Key         = '<Your Azure Key Here>'
Resource_Id = '/subscriptions/<Your subscription id here>/resourceGroups/<Your resource group>/providers/Microsoft.CognitiveServices/accounts/<Your account name>'
Project_Id  = '<Your project id>'

ACVObjectDetector = AzureCVObjectDetectionAPI(Endpoint, Key, Resource_Id, Project_Id)

In [ ]:
project_name = 'molecule_recognition_Saeki_one_labels'
ACVObjectDetector.create_project(project_name)

In [ ]:
ACVObjectDetector.upload_training_images(labeled_images)

In [ ]:
test_molecules_dir = './elsevier_images/test/image/'
test_labels_dir = './elsevier_images/train/label/'

test_images = []

for file in os.listdir(test_molecules_dir):
    if file.endswith(".jpeg"):
        image_path = os.path.join(test_molecules_dir, file)
        test_images.append(labeledImage(image_path))

In [ ]:
len(test_images)

In [ ]:
ACVObjectDetector.publish_iteration_name = 'Iteration1'

In [ ]:
ACVObjectDetector.predict_testing_images(test_images,probality_threshold=0.5)

In [ ]:
# save labels
test_labels_dir_prefix = './elsevier_images/test/'
test_labels_dir_affix  = '_labels'

for tag in labels:
    test_labels_dir = test_labels_dir_prefix + tag + test_labels_dir_affix
    if not os.path.exists(test_labels_dir):
        os.mkdir(test_labels_dir)

for img in test_images:
    for tag in img.labels.keys():
        test_labels_dir = test_labels_dir_prefix + tag + test_labels_dir_affix
        label_path = os.path.join(test_labels_dir, '_'.join(img.name.split('.')[:-1]) + '.txt')
        #print(label_path)
        img.write_labels_to_file(tag, label_path)

In [ ]:
# read saved_labels in
test_dir = './elsevier_images/test/image'
test_labels_dir = ['./elsevier_images/test/' + l + '_labels' for l in labels]

test_images = []

for file in os.listdir(test_dir):
    if file.endswith(".jpeg"):
        image_path = os.path.join(test_dir, file)
        test_images.append(labeledImage(image_path))
        
        for i in range(len(labels)):
            label_path = os.path.join(test_labels_dir[i], file.split('.')[0] + '.txt')
            if os.path.exists(label_path):
                #print(label_path)
                test_images[-1].add_labels_from_file(labels[i], label_path)

In [ ]:
test_images[0].labels

In [ ]:
# save train cropped files
croped_dir = './elsevier_images/cropped_molecules/train'
if not os.path.exists(croped_dir):
    os.mkdir(croped_dir)

for img in labeled_images:
    img.save_cropped_images_based_on_labels(croped_dir, saved_tags=['molecule'])

In [ ]:
# save test cropped files
croped_dir = './elsevier_images/cropped_molecules/test'
if not os.path.exists(croped_dir):
    os.mkdir(croped_dir)

test_image_scale = 1.2
for img in test_images:
    img.save_cropped_images_based_on_labels(croped_dir,  scale_bbox=test_image_scale, saved_tags=['molecule'])